In [54]:
from collections import defaultdict
import gc
from graphlib import TopologicalSorter, CycleError
import random
from typing import Collection, Any

import numpy as np
from scipy.sparse.csgraph import breadth_first_order, connected_components

from hostess.profilers import analyze_references

In [53]:
Refrec = dict[str, set]
Refgraph = dict[int, Refrec]

def blank_refrec() -> Refrec:
    return {'referents': set(), 'referrers': set(), 'cycles': set()} 

def get_refmap(
    objects: Collection[Any], 
    group_only: bool = True, 
    **analysis_kwargs
) -> dict[int, set[int]]:
    permit_ids = None if group_only is False else {id(obj) for obj in objects}
    for obj in objects:
        refnoms = analyze_references(
            method=gc.get_referents, 
            return_objects=False,
            permit_ids=permit_ids,
            **analysis_kwargs
        )
        refmap[id(obj)] = {rn[0]['id'] for rn in refnoms}
    return refmap

def _add_graph_referents(
    graph: Refgraph, refmap, delete_reflexive
) -> Refgraph:
    for objid, refids in refmap.items():
        graph[objid]['referents'].update(refids)
        if delete_reflexive is True:
            graph[objid]['referents'].difference_update({objid})
    return graph

def _add_graph_referrers(
    graph: Refgraph, delete_commutative
) -> Refgraph:
    # TODO, maybe: we _could_ create this using two-call
    #  referent-referrer detection, but that would be expensive and
    #  might not add much. assess.
    for objid in graph.keys():
        for refid in tuple(graph[objid]['referents']):
            if objid in graph[refid]['referents']:
                if delete_commutative is True:
                    graph[refid]['referents'].remove(objid)
                    graph[objid]['referents'].remove(refid)
                    continue
                else:
                    graph[refid]['cycles'].add(objid)
                    graph[objid]['cycles'].add(refid)
            graph[refid]['referrers'].add(objid)
    return graph

def construct_refgraph(
    objects: Collection[Any], 
    group_only=True,
    delete_reflexive=True,
    delete_commutative=False,
    **analysis_kwargs
) -> Refgraph:
    # NOTE: we are relying here entirely on what the gc reports. if an
    #  object is not tracked, we may want a fallback.
    refmap = get_refmap(objects, group_only, **analysis_kwargs)
    graph = defaultdict(blank_refrec)
    graph = _add_graph_referents(graph, refmap, delete_reflexive)
    graph = _add_graph_referrers(graph, delete_commutative)
    return dict(graph)
    
def random_test_refgraph(
    n=10, 
    group_only=True,
    delete_reflexive=True,
    delete_cyclic=False,
    density=1.5
) -> Refgraph:
    allids = tuple(range(n*10))  # dummy universe of all tracked objects
    objids = tuple(range(n))  # objects we care about references between 
    refmap = {}
    for objid in objids:
        refmap[objid] = set(random.choices(allids, k=int(density * n)))
        if group_only is True:
            refmap[objid].intersection_update(objids)
    graph = defaultdict(blank_refrec)
    graph = _add_graph_referents(graph, refmap, delete_reflexive)
    graph = _add_graph_referrers(graph, delete_cyclic)
    return dict(graph)

def verify_refgraph_directionality(graph: Refgraph) -> bool:
    for objid, rec in graph.items():
        for refid in rec['referents']:
            if objid not in graph[refid]['referrers']:
                return False
    return True

In [166]:
def refgraph_array(graph: Refgraph) -> np.ndarray:
    grapharray = np.zeros(
        (len(graph), len(graph)), dtype=np.uint16
    )
    for i, rec in enumerate(graph.values()):
        for j in rec['referents']:
            grapharray[i, j] = 1
    return grapharray

def refgraph_to_topdict(graph: Refgraph) -> dict[int, set]:
    return {id_: rec['referrers'] for id_, rec in graph.items()}

def refgraph_to_topsort(graph: Refgraph) -> TopologicalSorter:
    return TopologicalSorter(refgraph_to_topdict(graph))

def oksort(graph: np.ndarray) -> np.ndarray:
    """
    provide a sorting for a directed graph expressed as an array. 
    the graph need not be acyclic or connected. this is not 
    guaranteed to be an _optimal_ 'topological-esque' sorting, but is 
    guaranteed to not be the _worst_ 'topological-esque' sorting.
    internally-connected groups of nodes will always appear in sequence 
    in the final sorting; the way in which these groups are ordered
    is arbitrary.
    """
    sorting = []
    # sort each internally-connected cluster individually.
    # note that each cluster is wholly disconnected from 
    # each other cluster.
    _, labels = connected_components(graph)
    for label in np.unique(labels):
        cluster = np.nonzero(labels==label)[0]
        # check nodes in order of number of referents
        # until we run out of unwalked nodes
        refgroups = defaultdict(set)
        for node, count in zip(
            cluster, graph[cluster].sum(axis=1)
        ):
            refgroups[count].add(node)
        refgroups = list(refgroups.values())
        levels = {}
        while len(levels) < len(cluster):
            start = refgroups[-1].pop()
            walk = breadth_first_order(
                graph, start, return_predecessors=False
            )
            i = 0
            for node in walk:
                if node in levels.keys():
                    continue
                levels[node], i = i, i + 1
            if len(refgroups[-1]) == 0:
                refgroups.pop()
        # then, sort the nodes of the cluster by the position 
        # in which they first appear in any traversal; this is
        # an ordering for this set of connected nodes.
        sorting.append(
            sorted(levels.keys(), key=lambda k: levels[k])
        )
    # final ordering is the concatenation of the orderings of each
    # set of connected nodes.
    return tuple(np.concatenate(sorting))

In [273]:
gdict = random_test_refgraph(n=12, density=0.5)
sorter = refgraph_to_topsort(gdict)
try:
    top_sorting = tuple(sorter.static_order()) 
    # print(top_sorting)
except CycleError:
    top_sorting =None
    print('cyclic; no topological sort')
garray = refgraph_array(gdict)
ok_sorting = oksort(garray)
for index, node in enumerate(ok_sorting):
    for subsequent in ok_sorting[index + 1 :]:
        if node in gdict[subsequent]['referents']:
            print(f"{subsequent} succeeds {node} but {node} is its referent")

0 succeeds 8 but 8 is its referent
9 succeeds 8 but 8 is its referent
5 succeeds 4 but 4 is its referent


In [274]:
ok_sorting

(8, 0, 11, 9, 1, 7, 6, 2, 4, 3, 5, 10)

In [275]:
gdict

{0: {'referents': {8}, 'referrers': set(), 'cycles': set()},
 1: {'referents': {6}, 'referrers': {8}, 'cycles': set()},
 2: {'referents': set(), 'referrers': set(), 'cycles': set()},
 3: {'referents': {5}, 'referrers': set(), 'cycles': set()},
 4: {'referents': set(), 'referrers': {5}, 'cycles': set()},
 5: {'referents': {4}, 'referrers': {3}, 'cycles': set()},
 6: {'referents': set(), 'referrers': {1}, 'cycles': set()},
 7: {'referents': set(), 'referrers': {8, 11}, 'cycles': set()},
 8: {'referents': {1, 7}, 'referrers': {0, 9}, 'cycles': set()},
 9: {'referents': {8}, 'referrers': set(), 'cycles': set()},
 10: {'referents': set(), 'referrers': set(), 'cycles': set()},
 11: {'referents': {7}, 'referrers': set(), 'cycles': set()}}

In [ ]:
ok

In [ ]:
csgraph.breadth_first_order(garray, 3, return_predecessors=False)

In [ ]:
garray = np.array([[0,1,1],[0,0,0],[0,0,0]])
csgraph.breadth_first_order(garray, 2)

In [ ]:
a = [1, 2]
b = [1, a]
b in gc.get_referrers(a)

In [ ]:
# NOTE: we want to be able to place objects that refer to others first, 
# so that we do not replace containing objects with corrupt doppelgangers
# before checking _all_ their top-level objects.
def sort_refgraph(graph: dict[int, dict[str, set]]) -> list[int]:
    """
    provide a sorting for the elements of an acyclic graph such that no 
    element precedes an element that refers to it. 
    
    note that because this graph is not guaranteed to be either
    directed or connected, its edges do _not_ provide a total ordering.
    this means that the output of this function will satisfy the above 
    criterion but may vary based on the order of elements passed to it.
    """
    # TODO: evaluate efficiency. note that classical sorting algorithms
    #  don't tend to work well on partial orderings...
    n = 1
    seq = list(graph.keys()).copy()
    is_sorted = False
    indices = tuple(range(len(graph)))
    while is_sorted is False:
        # print(f"\n\n---cycle {n}---")
        is_sorted = True
        for key in seq:
            allowable_positions = []
            for j in indices[indices.index(key) + 1:]:
                ref = seq[j]
                if key in graph[ref]['referents']:
                    is_sorted = False
                    continue
                allowable_positions.append(j)
            if allowable_positions == []:
                swap_position = indices[-1]
            else:
                swap_position = allowable_positions[-1]
            left = seq[:swap_position]
            right = seq[swap_position + 1:]
            print(key, seq[swap_position], allowable_positions)
            seq = left + [key] + right
    return seq

In [ ]:
left, right = set(), set()
seq = list(testgraph.keys())
key = seq[0]
for ref in seq:
    if ref == key:
        continue
    if key in testgraph[ref]['referents']:
        left.add(ref)
    else:
        right.add(ref)

In [ ]:
keys = list(testgraph.keys())
key = keys[0]
left = testgraph[key]['referrers']
right = testgraph[key]['referents']
center = set(keys).difference(left.union(right))

In [ ]:
left

In [ ]:
right

In [ ]:
center

In [ ]:
testgraph

In [ ]:
sorted(testgraph, key=lambda i, j: testgraph[i] in testgraph[j]['referrers'])

In [ ]:
testgraph = random_test_refgraph(n=8, density=3, delete_cyclic=True)
t_refgraph_reflexivity(testgraph)
sorting = sort_refgraph(testgraph)
# TEST:
bad = {}
for i, objid in enumerate(sorting):
    for refid in sorting[i + 1:]:
        if objid in testgraph[refid]['referents']:
            bad[objid] = testgraph[objid]
            bad[refid] = testgraph[refid]
bad

In [ ]:
sorting

In [ ]:
bad

In [ ]:
%debug

In [ ]:
testgraph[5]

In [ ]:
5 in testgraph[7]['referrers']

In [ ]:
testgraph[7]

In [ ]:
{ref: refgraph[ref] for ref in refsorted}

In [ ]:
refgraph